In [24]:
import requests
import pandas as pd
from datetime import datetime

In [18]:
DAILY = 11
API_URL = 'https://www.nordpoolgroup.com/api/marketdata/page/%i'
r = requests.get(API_URL % DAILY, params={
            'currency': "SEK",
            'endDate': datetime.now().strftime("%d-%m-%Y"),
        })

currency = r.json()['currency']
data = r.json()['data']

In [20]:
start_time = data['DataStartdate']
end_time = data['DataEnddate']
updated = data['DateUpdated']

In [21]:
start_time

'2022-10-08T00:00:00'

In [25]:
def fetch_electricity_prices(historical=False):
    HOURLY = 10
    DAILY = 11
    API_URL = 'https://www.nordpoolgroup.com/api/marketdata/page/%i'
    r = requests.get(API_URL % DAILY, params={
                'currency': "SEK",
                'endDate': datetime.now().strftime("%d-%m-%Y"),
            })

    areas=['SE1', 'SE2', 'SE3', 'SE4']
    areas_data = {}
    areas_data[areas[0]] = {}

    currency = r.json()['currency']
    data = r.json()['data']
    start_time = data['DataStartdate']
    end_time = data['DataEnddate']
    updated = data['DateUpdated']

    area_price_list = []
    for r in data['Rows']:
        for c in r['Columns']:
            if c['Name'] in areas:
                area_price_list.append({"start_time": r["StartTime"], "end_time": r["EndTime"], "area": c["Name"], "price":c['Value']})
    pdf = pd.DataFrame(area_price_list)
    pdf["day"] = pdf["start_time"].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').strftime("%Y-%m-%d"))
    pdf = pdf.drop(["start_time", "end_time"], axis=1)
    pdf.price = pdf.price.map(lambda x: float(x.replace(" ", "").replace(",", ".")))
    pdf = pdf.groupby(["day", "area"]).agg({'price': ['mean']}).reset_index()
    pdf.columns = ["day", "area", "price"]
    if not historical:
        today = (date.today()).strftime("%Y-%m-%d")
        pdf = pdf[pdf.day == today]

    SE1 = pdf[pdf.area == "SE1"].drop(["area"], axis=1)
    SE1.columns = ["day", "price_se1"]
    SE2 = pdf[pdf.area == "SE2"].drop(["area"], axis=1)
    SE2.columns = ["day", "price_se2"]
    SE3 = pdf[pdf.area == "SE3"].drop(["area"], axis=1)
    SE3.columns = ["day", "price_se3"]
    SE4 = pdf[pdf.area == "SE4"].drop(["area"], axis=1)
    SE4.columns = ["day", "price_se4"]

    pdf = SE1.merge(SE2, on=["day"], how = "outer")
    pdf = pdf.merge(SE3, on=["day"], how = "outer")
    pdf = pdf.merge(SE4, on=["day"], how = "outer")

    pdf["timestamp"] = pdf["day"].map(lambda x: int(float(datetime.strptime(x, "%Y-%m-%d").timestamp()) * 1000))
    return pdf

In [26]:
df = fetch_electricity_prices(True)

In [27]:
df

,day,price_se1,price_se2,price_se3,price_se4,timestamp
0,2022-10-08,81.04,81.04,81.04,81.04,1665201600000
1,2022-10-09,125.66,125.66,125.66,125.66,1665288000000
2,2022-10-10,118.70,118.70,118.70,118.70,1665374400000
3,2022-10-11,98.91,98.91,98.91,98.91,1665460800000
4,2022-10-12,206.57,207.40,1407.31,1407.31,1665547200000
5,2022-10-13,296.50,296.50,983.66,983.66,1665633600000
6,2022-10-14,970.67,970.67,1660.38,1660.38,1665720000000
7,2022-10-15,211.18,211.18,354.88,354.88,1665806400000
8,2022-10-16,197.20,197.20,197.20,197.20,1665892800000
9,2022-10-17,261.77,261.77,873.78,873.78,1665979200000
